# GenAI with Python: Coding Agents

In [2]:
import pandas as pd

dtf = pd.read_csv("data_titanic.csv")
dtf.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [23]:
import sqlite3

dtf.to_sql(index=False, name="titanic", con=sqlite3.connect("db.db"), if_exists="replace")

891

In [17]:
from langchain_community.llms import Ollama #0.2.12

llm = Ollama(model="llama3.1")
res = llm.invoke(input=["How are you?"]).split("\n")[0]
print(res)

I don't have feelings or emotions like humans do, so I don't have a current state of being in the way that a person would. However, I'm functioning properly and ready to assist with any questions or tasks you may have! Is there something specific you'd like help with?


In [25]:
from crewai_tools import tool #0.8.3
from langchain_community.tools.sql_database.tool import ListSQLDatabaseTool, InfoSQLDatabaseTool
from langchain_community.utilities.sql_database import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///db.db")

@tool("list_tables")
def list_tables() -> str:
    """List the available tables in the database"""
    return ListSQLDatabaseTool(db=db).invoke("")

@tool("tables_schema")
def tables_schema(tables: str) -> str:
    """
    Input is a comma-separated list of tables, output is the schema and sample rows
    for those tables. Be sure that the tables actually exist by calling `list_tables` first!
    Example Input: table1, table2, table3
    """
    tool = InfoSQLDatabaseTool(db=db)
    return tool.invoke(tables)

print( tables_schema.run( tables=list_tables.run() ) )

Using Tool: list_tables
Using Tool: tables_schema

CREATE TABLE titanic (
	"PassengerId" INTEGER, 
	"Survived" INTEGER, 
	"Pclass" INTEGER, 
	"Name" TEXT, 
	"Sex" TEXT, 
	"Age" REAL, 
	"SibSp" INTEGER, 
	"Parch" INTEGER, 
	"Ticket" TEXT, 
	"Fare" REAL, 
	"Cabin" TEXT, 
	"Embarked" TEXT
)

/*
3 rows from titanic table:
PassengerId	Survived	Pclass	Name	Sex	Age	SibSp	Parch	Ticket	Fare	Cabin	Embarked
1	0	3	Braund, Mr. Owen Harris	male	22.0	1	0	A/5 21171	7.25	None	S
2	1	1	Cumings, Mrs. John Bradley (Florence Briggs Thayer)	female	38.0	1	0	PC 17599	71.2833	C85	C
3	1	3	Heikkinen, Miss. Laina	female	26.0	0	0	STON/O2. 3101282	7.925	None	S
*/


In [28]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

@tool("execute_sql")
def execute_sql(sql_query: str) -> str:
    """Execute a SQL query against the database. Returns the result"""
    return QuerySQLDataBaseTool(db=db).invoke(sql_query)

execute_sql.run(f"SELECT * FROM {list_tables.run()} LIMIT 3")

Using Tool: list_tables
Using Tool: execute_sql


"[(1, 0, 3, 'Braund, Mr. Owen Harris', 'male', 22.0, 1, 0, 'A/5 21171', 7.25, None, 'S'), (2, 1, 1, 'Cumings, Mrs. John Bradley (Florence Briggs Thayer)', 'female', 38.0, 1, 0, 'PC 17599', 71.2833, 'C85', 'C'), (3, 1, 3, 'Heikkinen, Miss. Laina', 'female', 26.0, 0, 0, 'STON/O2. 3101282', 7.925, None, 'S')]"

In [30]:
from langchain_community.tools.sql_database.tool import QuerySQLCheckerTool
from langchain_community.llms import Ollama #0.2.12

llm = Ollama(model="llama3.1")

@tool("check_sql")
def check_sql(sql_query: str) -> str:
    """
    Use this tool to double check if your query is correct before executing it. Always use this
    tool before executing a query with `execute_sql`.
    """
    return QuerySQLCheckerTool(db=db, llm=llm).invoke({"query": sql_query})

check_sql.run(f"SELECT * FROM {list_tables.run()} LIMIT 3 +db+")

Using Tool: list_tables
Using Tool: check_sql


'SELECT * FROM titanic LIMIT 3'

In [37]:
import crewai #0.51.1

prompt = '''Construct and execute SQL queries based on {user_input}'''

## Agent
agent_sql = crewai.Agent(
    role="Database Developer",
    goal=prompt,
    backstory='''
        You are an experienced database engineer who is master at creating efficient and complex SQL queries.
        You have a deep understanding of how different databases work and how to optimize queries.
        Use the `list_tables` to find available tables.
        Use the `tables_schema` to understand the metadata for the tables.
        Use the `execute_sql` to check your queries for correctness.
        Use the `check_sql` to execute queries against the database.
     ''',
    tools=[list_tables, tables_schema, execute_sql, check_sql], 
    max_iter=3,
    llm=llm,
    allow_delegation=False, verbose=True)

## Task
task_sql = crewai.Task(
    description=prompt,
    agent=agent_sql,
    expected_output='''Database result for the query''')

In [38]:
prompt = '''You receive data from the database developer and analyze it to answer {user_input}'''

## Agent
agent_python = crewai.Agent(
    role="Senior Data Analyst",
    goal=prompt,
    backstory='''
        You have deep experience with analyzing datasets using Python.
        Your work is always based on the provided data and is clear,
        easy-to-understand and to the point. You have attention
        to detail and always produce very detailed work.
     ''',
    #tools=[], 
    max_iter=3,
    llm=llm,
    allow_delegation=False, verbose=True)

## Task
task_python = crewai.Task(
    description=prompt,
    agent=agent_python,
    expected_output='''Detailed analysis text''')

In [39]:
crew = crewai.Crew(agents=[agent_sql, agent_python], 
                   tasks=[task_sql, task_python], 
                   process=crewai.Process.sequential,
                   verbose=True)

res = crew.kickoff(inputs={"user_input":"how many people died?"})

print("\n---Res---\n", res)

2024-08-26 00:45:01,748 - 140704473259456 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed


 [2024-08-26 00:45:01][DEBUG]: == Working Agent: Database Developer
 [2024-08-26 00:45:01][INFO]: == Starting Task: Construct and execute SQL queries based on how many people died?


> Entering new CrewAgentExecutor chain...
Thought: I need to know what tables are available in the database.
Action: list_tables
Action Input: 

titanic

Thought:
Thought: Now that I know what tables are available, I should check the schema for each table to see which ones have information about people who died.
Action: tables_schema
Action Input: {'tables': "titanic 


CREATE TABLE titanic (
	"PassengerId" INTEGER, 
	"Survived" INTEGER, 
	"Pclass" INTEGER, 
	"Name" TEXT, 
	"Sex" TEXT, 
	"Age" REAL, 
	"SibSp" INTEGER, 
	"Parch" INTEGER, 
	"Ticket" TEXT, 
	"Fare" REAL, 
	"Cabin" TEXT, 
	"Embarked" TEXT
)

/*
3 rows from titanic table:
PassengerId	Survived	Pclass	Name	Sex	Age	SibSp	Parch	Ticket	Fare	Cabin	Embarked
1	0	3	Braund, Mr. Owen Harris	male	22.0	1	0	A/5 21171	7.25	None	S
2	1	1	Cumings, Mrs. John Brad

TypeError: can only concatenate str (not "CrewOutput") to str